In [1]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
class NUSW_NB15_Dataset(Dataset):
    def __init__(self, path):
        ds_type = path.split('/')[-1].split('-')[0]
        df = pd.read_csv(path)
        
        x = df.drop(['id', 'attack_cat', 'label'], axis=1)
        y = df['label']

        self.x = torch.Tensor(x.to_numpy())
        self.y = torch.Tensor(y).to(dtype=torch.long)

        self.dim = self.x.shape[1]

        print(
            f'Finished reading the {ds_type} set of Dataset '\
            f'({len(self.x)} samples found, each dim = {self.dim})'
        )

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)

In [3]:
def prep_dataloader(path, batch_size, shuffle):
    dataset = NUSW_NB15_Dataset(path)
    dataloader = DataLoader(
        dataset,
        batch_size,
        shuffle
    )
    return dataloader

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(196, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    avg_loss, avg_correct = 0, 0
    model.train()
    with tqdm(dataloader) as pbar:
        for (X, y) in pbar:
            X, y = X.to(device), y.to(device)
            
            # Compute prediction error
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # sum loss and correct
            avg_loss += loss.item()
            avg_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
            # set loss val to pbar
            pbar.set_postfix(loss=f'{loss.item():>7f}')
    
    # cal avg loss and correct
    avg_loss /= num_batches
    avg_correct /= size
    print(f"[Train] Accuracy: {(100*avg_correct):>0.1f}%, Avg loss: {avg_loss:>8f} \n")
    return avg_loss, avg_correct

In [6]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"[Test] Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss, correct

In [7]:
# path
tr_path = '../data/processed/training-set.csv'
val_path = '../data/processed/validation-set.csv'
tt_path = '../data/processed/testing-set.csv'
log_path = '../logs/anomaly_detection'
model_path = '../models/anomaly_detection/model_weights.pth'

# hyperparameter
batch_size = 64

In [8]:
# prepare dataloader
tr_dl = prep_dataloader(
    tr_path,
    batch_size,
    shuffle=True
)

val_dl = prep_dataloader(
    val_path,
    batch_size,
    shuffle=True
)

tt_dl = prep_dataloader(
    tt_path,
    batch_size,
    shuffle=False
)

Finished reading the training set of Dataset (164910 samples found, each dim = 196)
Finished reading the validation set of Dataset (41228 samples found, each dim = 196)
Finished reading the testing set of Dataset (51535 samples found, each dim = 196)


In [9]:
# device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# summary writer
last_log_path = os.path.join(log_path, datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
writer = SummaryWriter(last_log_path)

# neural network
model = NeuralNetwork().to(device)

# load model weight
if os.path.isfile(model_path):
    print(f'Load model weights form {model_path}')
    model.load_state_dict(torch.load(model_path))

# loss func and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# training and validation
epochs = 2
for t in range(epochs):
    ep = t + 1
    print(f"Epoch {ep}")
    tr_loss, tr_acc = train(tr_dl, model, loss_fn, optimizer)
    tt_loss, tt_acc = test(val_dl, model, loss_fn)
    
    # log loss and acc
    writer.add_scalar('Loss/train', tr_loss, ep)
    writer.add_scalar('Accuracy/train', tr_acc, ep)
    writer.add_scalar('Loss/test', tt_loss, ep)
    writer.add_scalar('Accuracy/test', tt_acc, ep)
    
print(f'Saving model to {model_path}\n')
torch.save(model.state_dict(), model_path)
    
print("Done!")

Using cuda device
Load model weights form ../models/anomaly_detection/model_weights.pth
Epoch 1


  0%|          | 0/2577 [00:00<?, ?it/s]

[Train] Accuracy: 89.3%, Avg loss: 0.237368 

[Test] Accuracy: 89.4%, Avg loss: 0.227329 

Epoch 2


  0%|          | 0/2577 [00:00<?, ?it/s]

[Train] Accuracy: 89.6%, Avg loss: 0.220199 

[Test] Accuracy: 89.7%, Avg loss: 0.213946 

Saving model to ../models/anomaly_detection/model_weights.pth

Done!


In [10]:
# testing
test(tt_dl, model, loss_fn)

[Test] Accuracy: 89.4%, Avg loss: 0.218754 



(0.21875412857377205, 0.8938003298729019)